# Preliminaries

Check for GPU

In [1]:
import tensorflow as tf

In [2]:
if tf.test.gpu_device_name() != '/device:GPU:0':
  print("No GPU found")
else:
  print("GPU ready: {}".format(tf.test.gpu_device_name()))

No GPU found


Import preprocessing helper functions

In [3]:
import sys
sys.path.append("../../helper-modules")
from preprocessing_utils import read_in_data, preprocess

Read in the data to df_train, df_val and df_test

In [4]:
df_train, df_val, df_test = read_in_data()

Create X_train, y_train, X_val, y_val, X_test, y_test

In [5]:
(X_train, y_train), (X_val, y_val), (X_test, y_test) = preprocess("CNN", df_train, df_test, df_val)

# Model Building

Imports

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, AvgPool2D, Flatten, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from itertools import product
import time
import matplotlib.pyplot as plt

Enable seed setting for improved reproducibility

In [7]:
import os
import random
import numpy as np

In [8]:
def set_seed(seed=42):
  SEED=seed

  # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
  os.environ['PYTHONHASHSEED']=str(SEED)

  # 2. Set `python` built-in pseudo-random generator at a fixed value
  random.seed(SEED)

  # 3. Set `numpy` pseudo-random generator at a fixed value
  np.random.seed(SEED)

  # 4. Set the `tensorflow` pseudo-random generator at a fixed value
  tf.random.set_seed(SEED)

Extract number of classes (14 classes)

In [9]:
NUM_CLASSES = df_test["label"].nunique() 

Model design

In [10]:
def compile_model(num_filters, filter_size = 3, padding_type="same", act_function="relu", pooling_type="max", dense_layer_size=16, dropout_rate=0.1, l_rate=0.001, reg='l2', opt_algo='adam'):
    """
    Compiles a shallow 2D-CNN model with the given hyperparameters.
    
    Architecture:
    
    Conv layer -> Pooling layer ->  Fully connected layer -> Softmax output layer
    
    Hyperparameter options:
    
    num_filters_tuple : An int with the number of filters for the conv layer
    filter_size : an int with the size of the filters (3 -> 3 x 3 filter)
    padding_type : either "same" or "valid"
    act_function : options include "relu", "sigmoid" and "tanh"
    pooling_type : either "max" or "avg"
    dense_layer_size : an int with the number of neurons in the fully connected layer
    dropout_rate : a float with the rate of dropout to be applied to the fully connected layer
    l_rate : a float with the learning rate
    reg : string with the regularization type e.g. "l2"
    opt_algo : the optimization algorithm, "adam" or "sgd"
    """  
    model = Sequential()

    # CONVOLUTIONAL LAYER 1
    model.add(Conv2D(filters=num_filters, kernel_size=(filter_size,filter_size), input_shape=(40, 37, 1), padding=padding_type))
    model.add(Activation(act_function))

    # POOLING LAYER 1
    if pooling_type == "avg":  # strides defaults to pool_size
      model.add(AvgPool2D(pool_size=(2, 2)))
    elif pooling_type == "max":
      model.add(MaxPool2D(pool_size=(2, 2)))
    else:
      raise Exception("Invalid pooling option entered")
      
    # FLATTEN OUTPUT
    model.add(Flatten())

    # # FULLY CONNECTED LAYER
    model.add(Dense(dense_layer_size, activation=act_function))

    # # DROPOUT to prevent overfitting
    model.add(Dropout(dropout_rate))
    
    # LAST LAYER IS THE CLASSIFIER, THUS 12 POSSIBLE CLASSES
    model.add(Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=reg))

    # Optimization algorithm
    if opt_algo == 'adam':
      opt = tf.keras.optimizers.Adam(learning_rate=l_rate)
    elif opt_algo == 'sgd':
      opt = tf.keras.optimizers.SGD(learning_rate=l_rate)

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

# Experiment: Grid Search

Defining grid search hyperparameter options

In [11]:
num_filters_ops = [4, 8, 16, 32, 64, 128]
l_rates = [0.01, 0.005, 0.001, 0.0005, 0.0001]
dropout_rates = [0.05, 0.1, 0.2, 0.5]
pooling_types = ["max"]
filter_sizes = [3]
padding_types = ["same"]
act_functions=["relu"]
regs = ['l2']
opt_algos = ['adam']

all_hparams = [num_filters_ops, l_rates, dropout_rates, pooling_types, filter_sizes, padding_types, act_functions, regs, opt_algos]

Conduct the grid search

In [12]:
def grid_search():
  """
  For each `num_filters_tuples` option (corresponding to a given number of parameters), conduct a grid search
  over the hyperparameter options, writing the validation results and model details for each model to a file.
  
  All trained models are saved as h5 files. 
  """
  best_model = None
  best_accuracy = 0
  best_history = None

  EPOCHS = 50
  BATCH_SIZE = 32
  STOPPING_PATIENCE = 5

  STARTING_POINT = 1

  with open('../training-results/ExperimentLogs_Shallow-CNN.csv', 'w') as log_file:
      log_file.write("ModelNumber;Timestamp;NumEpochs;ValAccuracy;NumParams;NumFiltersPerLayer;DenseLayerSize;LearningRate;DropoutRate;PoolingType;FilterSize;PaddingType;ActivationFunction;Regularization;OptimizationAlgorithm\n")

  for model_num, hparam_set in enumerate(list(product(*all_hparams))[(STARTING_POINT-1):], start=STARTING_POINT):
    # Extract hyperparams for current model from grid search grid
    num_filters, l_rate, dropout_rate, pooling_type, filter_size, padding_type, act_function, reg, opt_algo = hparam_set
    dense_layer_size = max(round(num_filters / 3), 2)  # set number of dense layer neurons to number of filters of first two conv layers

    # Define hyperparameters for the current model 
    hparams = {
        "num_filters" : num_filters,
        "filter_size" : filter_size, 
        "padding_type" : padding_type, 
        "act_function" : act_function, 
        "pooling_type" : pooling_type, 
        "dense_layer_size" : dense_layer_size, 
        "dropout_rate" : dropout_rate, 
        "l_rate": l_rate,
        "reg" : reg,
        "opt_algo" : opt_algo
    }

    # Compile model and count number of parameters
    model = compile_model(**hparams)
    num_params = model.count_params()

    model_str = f"{num_filters}; {dense_layer_size}; {l_rate}; {dropout_rate}; {pooling_type}; {filter_size}; {padding_type}; {act_function}; {reg}; {opt_algo}"
    print(f"Training model {model_num} with {num_params} params - {model_str}")

    # Prevent overfitting with early stopping
    early_stop = EarlyStopping(monitor='val_accuracy', patience=STOPPING_PATIENCE)
    # For saving best val accuracy model as an h5 (for test predictions later)
    model_check = ModelCheckpoint(f"../trained-models/Shallow-CNN_{model_num}.h5", monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)

    # Set seed for reproducibility
    set_seed(42)

    # Train model
    # Note: one reason val accuracy might be higher than train accuracy during training is because dropout affects training but not validation
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), callbacks=[early_stop, model_check])
    n_epochs = len(history.history['loss'])
                   
    # Time stamp when model finished training
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    # Extract best validation accuracy at inex 1 (index 0 has the loss)
    val_accuracy = max(history.history['val_accuracy'])
    print(f"Best val accuracy: {val_accuracy}")

    # Store results in log file
    with open('../training-results/ExperimentLogs_Shallow-CNN.csv', 'a') as log_file:
      log_file.write(f"{model_num};{timestamp};{n_epochs};{val_accuracy};{num_params};{model_str}\n")

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      best_model = model
      best_history = history

  # Return the model with the highest validation accuracy
  return best_model, best_history


In [ ]:
best_model, best_history = grid_search()

Training model 1 with 2961 params - 4; 2; 0.01; 0.05; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 6s 2ms/step - loss: 2.5979 - accuracy: 0.0747 - val_loss: 2.5984 - val_accuracy: 0.0769
Epoch 2/50
2600/2600 [==============================] - 6s 2ms/step - loss: 2.5976 - accuracy: 0.0758 - val_loss: 2.5982 - val_accuracy: 0.0769
Epoch 3/50
2600/2600 [==============================] - 6s 2ms/step - loss: 2.5977 - accuracy: 0.0763 - val_loss: 2.5974 - val_accuracy: 0.0769
Epoch 4/50
2600/2600 [==============================] - 5s 2ms/step - loss: 2.5977 - accuracy: 0.0761 - val_loss: 2.5972 - val_accuracy: 0.0769
Epoch 5/50
2600/2600 [==============================] - 5s 2ms/step - loss: 2.5976 - accuracy: 0.0766 - val_loss: 2.5976 - val_accuracy: 0.0769
Epoch 6/50
2600/2600 [==============================] - 5s 2ms/step - loss: 2.5978 - accuracy: 0.0769 - val_loss: 2.5970 - val_accuracy: 0.0769
Best val accuracy: 0.07692307978868484
Training model

Epoch 10/50
2600/2600 [==============================] - 6s 2ms/step - loss: 3.1875 - accuracy: 0.1529 - val_loss: 2.7231 - val_accuracy: 0.2148
Epoch 11/50
2600/2600 [==============================] - 6s 2ms/step - loss: 3.3144 - accuracy: 0.1520 - val_loss: 5.2563 - val_accuracy: 0.1314
Epoch 12/50
2600/2600 [==============================] - 6s 2ms/step - loss: 3.4917 - accuracy: 0.1475 - val_loss: 3.0329 - val_accuracy: 0.1967
Epoch 13/50
2600/2600 [==============================] - 6s 2ms/step - loss: 3.8830 - accuracy: 0.1481 - val_loss: 2.8216 - val_accuracy: 0.1015
Epoch 14/50
2600/2600 [==============================] - 6s 2ms/step - loss: 3.5897 - accuracy: 0.1431 - val_loss: 4.9020 - val_accuracy: 0.1673
Epoch 15/50
2600/2600 [==============================] - 6s 2ms/step - loss: 4.9036 - accuracy: 0.1377 - val_loss: 4.1656 - val_accuracy: 0.1496
Best val accuracy: 0.2148076891899109
Training model 12 with 2961 params - 4; 2; 0.001; 0.5; max; 3; same; relu; l2; adam
Epoch 1/

Epoch 1/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.3940 - accuracy: 0.1560 - val_loss: 2.1843 - val_accuracy: 0.1882
Epoch 2/50
2600/2600 [==============================] - 9s 4ms/step - loss: 2.4141 - accuracy: 0.1826 - val_loss: 2.3256 - val_accuracy: 0.2452
Epoch 3/50
2600/2600 [==============================] - 9s 4ms/step - loss: 2.5063 - accuracy: 0.1889 - val_loss: 2.4515 - val_accuracy: 0.1429
Epoch 4/50
2600/2600 [==============================] - 9s 4ms/step - loss: 2.5410 - accuracy: 0.1851 - val_loss: 2.9975 - val_accuracy: 0.1248
Epoch 5/50
2600/2600 [==============================] - 1004s 386ms/step - loss: 3.0420 - accuracy: 0.1454 - val_loss: 3.2246 - val_accuracy: 0.1505
Epoch 6/50
2600/2600 [==============================] - 1076s 414ms/step - loss: 4.1593 - accuracy: 0.1409 - val_loss: 4.2334 - val_accuracy: 0.1541
Epoch 7/50
2600/2600 [==============================] - 1925s 741ms/step - loss: 6.2184 - accuracy: 0.1436 - val_loss: 17.269

2600/2600 [==============================] - 1028s 395ms/step - loss: 2.9259 - accuracy: 0.2909 - val_loss: 1.9611 - val_accuracy: 0.3115
Epoch 4/50
2600/2600 [==============================] - 9s 3ms/step - loss: 2.2962 - accuracy: 0.2853 - val_loss: 2.2527 - val_accuracy: 0.3012
Epoch 5/50
2600/2600 [==============================] - 9s 3ms/step - loss: 2.2258 - accuracy: 0.2876 - val_loss: 2.0582 - val_accuracy: 0.3161
Epoch 6/50
2600/2600 [==============================] - 9s 3ms/step - loss: 2.2163 - accuracy: 0.2916 - val_loss: 2.1118 - val_accuracy: 0.3579
Best val accuracy: 0.41634616255760193
Training model 35 with 8775 params - 8; 3; 0.0005; 0.2; max; 3; same; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 10s 4ms/step - loss: 2.7749 - accuracy: 0.1203 - val_loss: 4.5033 - val_accuracy: 0.1724
Epoch 2/50
2600/2600 [==============================] - 8s 3ms/step - loss: 4.0589 - accuracy: 0.1428 - val_loss: 2.6249 - val_accuracy: 0.2169
Epoch 3/50
2600/2

# Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

### Training and validation performances over epochs

In [ ]:
plt.plot(best_history.history['loss'])
plt.plot(best_history.history['val_loss'])
plt.title('Convergence of the loss function')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train','val'])

In [ ]:
plt.plot(best_history.history['accuracy'])
plt.plot(best_history.history['val_accuracy'])
plt.title('Convergence of the accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train','val'])

### Confusion matrix

In [ ]:
plot_conf_mtx(best_model, X_val, y_val, df_val)